In [1]:
import psycopg2
import logging

In [2]:


def get_simpleplay_version():    
    try:
        # Define connection
        conn = psycopg2.connect("host=172.17.0.1 dbname=collectorDB user=postgres port=5432", password='secret')
        cursor = conn.cursor()

        # Execute Query
        cursor.execute("SELECT row_version FROM pgsoft_version LIMIT 1")

        # Get all results
        pgsoft_version = cursor.fetchone()

        print(f"====================== PG_SOFT Version {pgsoft_version[0]} ======================")
        cursor.close()
        conn.close()
            
        return pgsoft_version[0]
    
    except:
        logging.fatal("Unable to fetch data") 

In [4]:
temp = get_simpleplay_version()
temp

====================== PG_SOFT Version 4 ======================


4